# **Vehicle Verification - Lab 3 Acas Xu**


## Reference

Vehicle
- Tutorial: https://vehicle-lang.github.io/tutorial/
- Repo: https://github.com/vehicle-lang/vehicle
- Installation: https://vehicle-lang.readthedocs.io/en/latest/installation.html
- Full documentation: https://vehicle-lang.readthedocs.io/en/latest/index.html

ACAS Xu
- https://github.com/vehicle-lang/vehicle/tree/dev/examples/acasXu
- Katz, G., Barrett, C., Dill, D. L., Julian, K., & Kochenderfer, M. J. (2017). Reluplex: An efficient SMT solver for verifying deep neural networks. International Conference on Computer Aided Verification, 97–117.

Marabou
- Katz, G., Huang, D. A., Ibeling, D., Julian, K., Lazarus, C., Lim, R., Shah, P., Thakoor, S., Wu, H., Zeljić, A., & others. (2019). The Marabou framework for verification and analysis of deep neural networks. International Conference on Computer Aided Verification, 443–452.


***
# Vehicle - ACAS Xu Properties

## Imports

In [1]:
# Local Functions Import
#

# Hack to get local imports from src
import sys
from pathlib import Path

src_folder = Path.cwd().parent.joinpath('src')
if not src_folder.exists():
    raise FileNotFoundError(f'{src_folder} does not exist')
sys.path.insert(0, str(src_folder.resolve()))
del src_folder

# Local Functions
from data import run_setup

In [2]:
# Package imports
#

# General
import os
from datetime import datetime
import time
import sys
import platform
import shutil

# Other
import pandas as pd
import numpy as np

# Vehicle specific
import vehicle_lang
from vehicle_lang import Verifier
import onnx, onnxruntime
from maraboupy import Marabou


Instructions for updating:
non-resource variables are not supported in the long term


/opt/anaconda3/envs/Vehicle/lib/python3.11/site-packages/maraboupy/Marabou.py:31: UserWarning: ONNX parser is unavailable because onnx or onnxruntime packages are not installed
  warnings.warn("ONNX parser is unavailable because onnx or onnxruntime packages are not installed")


In [3]:
# Check Installs
#

print("System Information:")
print("=" * 100)
print(f"Python version: {sys.version}")
print(f"Platform: {platform.platform()}")
print(f"Architecture: {platform.architecture()}")
print()

conda_env = os.environ.get('CONDA_DEFAULT_ENV', 'Unknown')
print(f"Conda Environment: {conda_env}")
print()

!pip list | grep -E "(pandas|numpy|matplotlib|scikit-learn|tensorflow|tensorflow-metal|keras-tuner)"
!pip list | grep -E "(vehicle|marabou|onnx|onnxruntime)"

System Information:
Python version: 3.11.11 | packaged by conda-forge | (main, Dec  5 2024, 08:47:03) [Clang 18.1.8 ]
Platform: macOS-15.6-arm64-arm-64bit
Architecture: ('64bit', '')

Conda Environment: Vehicle

matplotlib-inline       0.2.1
numpy                   2.3.4
pandas                  2.3.3
tensorflow              2.18.0
tensorflow_estimator    2.15.0
maraboupy               1.0.0
onnx                    1.17.0
onnxruntime             1.22.0
vehicle_lang            0.22.0


## Runs

In [4]:
# Run Setup
#

run_name = 'Vehicle_Test'
local_project_folder = Path.cwd().parent
data_folder, run_results_folder = run_setup(local_project_folder, run_name)

In [5]:
# Simple Function to Type Check the Vehicle Script
# return empty list if all ok
#

def vehicle_check(folder, specification):

    specification = data_folder.joinpath(specification)

    result = !vehicle check \
        --specification {specification} \
        
    return result


In [6]:
# Simple Function to Run the Vehicle Script
#

def vehicle_verify(folder, specification, verifier, nn_model, properties):

    specification = folder.joinpath(specification)
    nn_model =  folder.joinpath(nn_model)

    result = !vehicle verify \
        --specification {specification} \
        --verifier {verifier} \
        --network acasXu:{nn_model} \
        --property {properties}

    return result

## Example - Property 3

In [ ]:
# Tutorial Given Example for Property 3
#

spec = 'acasXu.vcl'
verifier = 'Marabou'
models = ['acasXu_1_7.onnx', 'acasXu_1_8.onnx','acasXu_1_9.onnx']
spec_property = 'property3'

for model in models:
    verification_results = vehicle_verify(data_folder, spec, verifier, model, spec_property)
    print(f'Verification Results for: {spec} with model: {model}')
    print('\n'.join(verification_results))

## Extended - Properties

In [7]:
# Type Check the spec
#

spec = 'acasXu_extended.vcl'

check_result = vehicle_check(data_folder, spec)
print('Check OK' if not check_result else '\n'.join(check_result))


Check OK


In [8]:
# Additional Properties
#

spec = 'acasXu_extended.vcl'
spec_properties = ['property1', 'property3', 'property5']
verifier = 'Marabou'
models = ['acasXu_1_7.onnx', 'acasXu_1_8.onnx','acasXu_1_9.onnx']
# models = ['acasXu_1_7.onnx']

for spec_property in spec_properties:
    for model in models:
        verification_results = vehicle_verify(data_folder, spec, verifier, model, spec_property)
        print()
        print("=" * 100)
        print(f'Verification Results for: {spec}, {spec_property} with model: {model}')
        print('\n'.join(verification_results))



Verification Results for: acasXu_extended.vcl, property1 with model: acasXu_1_7.onnx


In order to provide support, Vehicle has automatically converted the strict inequalities to non-strict inequalites. This is not sound, but errors will be at most the floating point epsilon used by the verifier, which is usually very small (e.g. 1e-9). However, this may lead to unexpected behaviour (e.g. loss of the law of excluded middle).

See https://github.com/vehicle-lang/vehicle/issues/74 for further details.

Verifying properties:
  property1 [......................................................] 0/1 queries
  property1 [======================================================] 1/1 queries
    result: 🗸 - Marabou proved no counterexample exists

Verification Results for: acasXu_extended.vcl, property1 with model: acasXu_1_8.onnx


In order to provide support, Vehicle has automatically converted the strict inequalities to non-strict inequalites. This is not sound, but errors will be at most the